In [9]:
import pandas as pd

# ============================================================================
# CONFIGURATION
# ============================================================================
SUMMARY_FILE = "folddisco_summary.csv"  # Input summary file
OUTPUT_SOLUBILITY_UP = "solubility_up_summary.csv"
OUTPUT_SOLUBILITY_DOWN = "solubility_down_summary.csv"

# Dataset labels (should match what you used in the previous notebook)
SOLUBILITY_UP_LABEL = "solubility_up"  # Change to match your actual label
SOLUBILITY_DOWN_LABEL = "solubility_down"  # Change to match your actual label

# ============================================================================
# LOAD DATA
# ============================================================================

print("Loading folddisco summary...")
df_summary = pd.read_csv(SUMMARY_FILE)
print(f"✓ Loaded {len(df_summary)} rows\n")

print("Dataset breakdown:")
print(df_summary['dataset'].value_counts())
print()

# ============================================================================
# AUGMENT WITH COUNTS
# ============================================================================

print("Calculating total result counts per query (excluding self-matches)...")

# Calculate counts per query protein
query_stats = []

for dataset in df_summary['dataset'].unique():
    dataset_df = df_summary[df_summary['dataset'] == dataset]
    
    for query_uniprot in dataset_df['query_uniprot_id'].unique():
        query_df = dataset_df[dataset_df['query_uniprot_id'] == query_uniprot]
        
        # Count unique targets, excluding self-matches
        unique_targets_count = query_df[
            query_df['target_uniprot_id'] != query_uniprot
        ]['target_uniprot_id'].nunique()
        
        # Get list of target IDs (excluding self)
        target_ids_list = query_df[
            query_df['target_uniprot_id'] != query_uniprot
        ]['target_uniprot_id'].unique().tolist()
        
        query_stats.append({
            'dataset': dataset,
            'query_uniprot_id': query_uniprot,
            'unique_targets_count': unique_targets_count,
            'target_ids': ', '.join(target_ids_list)  # Comma-separated list
        })

df_query_stats = pd.DataFrame(query_stats)

print(f"✓ Calculated stats for {len(df_query_stats)} query proteins\n")

# ============================================================================
# SPLIT BY DATASET AND SORT
# ============================================================================

print("Splitting by dataset and sorting by target counts...")

# Solubility Up
df_solubility_up = df_query_stats[
    df_query_stats['dataset'] == SOLUBILITY_UP_LABEL
].copy()

df_solubility_up = df_solubility_up.sort_values(
    'unique_targets_count',
    ascending=False
).reset_index(drop=True)

# Solubility Down
df_solubility_down = df_query_stats[
    df_query_stats['dataset'] == SOLUBILITY_DOWN_LABEL
].copy()

df_solubility_down = df_solubility_down.sort_values(
    'unique_targets_count',
    ascending=False
).reset_index(drop=True)

# ============================================================================
# SAVE TO CSV
# ============================================================================

print(f"\nSaving datasets...")

df_solubility_up.to_csv(OUTPUT_SOLUBILITY_UP, index=False)
print(f"✓ Solubility Up saved to: {OUTPUT_SOLUBILITY_UP}")
print(f"  Rows: {len(df_solubility_up)}")
print(f"  Unique queries: {df_solubility_up['query_uniprot_id'].nunique()}")

df_solubility_down.to_csv(OUTPUT_SOLUBILITY_DOWN, index=False)
print(f"✓ Solubility Down saved to: {OUTPUT_SOLUBILITY_DOWN}")
print(f"  Rows: {len(df_solubility_down)}")
print(f"  Unique queries: {df_solubility_down['query_uniprot_id'].nunique()}")

# ============================================================================
# DISPLAY SUMMARY STATISTICS
# ============================================================================

print("\n" + "="*70)
print("SOLUBILITY UP - TOP 10 QUERIES BY TARGET COUNT")
print("="*70)
print(df_solubility_up.head(10).to_string(index=False))

print("\n" + "="*70)
print("SOLUBILITY DOWN - TOP 10 QUERIES BY TARGET COUNT")
print("="*70)
print(df_solubility_down.head(10).to_string(index=False))

print("\n✓ Complete! DataFrames available as: df_solubility_up, df_solubility_down")

Loading folddisco summary...
✓ Loaded 1935 rows

Dataset breakdown:
dataset
solubility_down    1167
solubility_up       768
Name: count, dtype: int64

Calculating total result counts per query (excluding self-matches)...
✓ Calculated stats for 452 query proteins

Splitting by dataset and sorting by target counts...

Saving datasets...
✓ Solubility Up saved to: solubility_up_summary.csv
  Rows: 259
  Unique queries: 259
✓ Solubility Down saved to: solubility_down_summary.csv
  Rows: 193
  Unique queries: 193

SOLUBILITY UP - TOP 10 QUERIES BY TARGET COUNT
      dataset query_uniprot_id  unique_targets_count                                     target_ids
solubility_up           P07759                     6 P61939, Q00898, Q00896, P07758, Q00897, P22599
solubility_up           P07758                     6 Q00896, Q00898, P22599, Q00897, P61939, P07759
solubility_up           Q00896                     6 P07758, Q00898, Q00897, P22599, P61939, P07759
solubility_up           Q00897         